In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [2]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [3]:
##Load NER model
model_ner=spacy.load('./Output/model-best/')

C:\Users\VENUS\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.6 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
#Load image 
image=cv2.imread('./data/6.jpg')
# cv2.imshow('businesscard',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows


# Extract data from image using pytessearct 
tessData=pytesseract.image_to_data(image)
#convert into dataframe
tessList=list(map(lambda x:x.split('\t'), tessData.split('\n')))
df=pd.DataFrame(tessList[1:],columns=tessList[0])


# Convert data into content # get predictions from NER model


In [11]:
df.dropna(inplace=True)

In [13]:
df['text']=df['text'].apply(cleanText)

In [16]:
df_clean=df.query('text!= "" ')
content=" ".join([w for w in df_clean['text']])

In [18]:
content

'cell 8099948528 te 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail .com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo'

In [19]:
doc=model_ner(content)

In [22]:
from spacy import displacy

In [23]:
displacy.serve(doc,style='ent')

C:\Users\VENUS\anaconda3\lib\site-packages\spacy\displacy\__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [25]:
displacy.render(doc,style='ent')

# Tagging

In [27]:
docjson=doc.to_json()

In [28]:
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [29]:
doc_text=docjson['text']


In [31]:
datafram_token=pd.DataFrame(docjson['tokens'])

In [32]:
datafram_token['token']=datafram_token[['start','end']].apply(
lambda x:doc_text[x[0]:x[1]],axis=1)

In [33]:
datafram_token.head(10)

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,te
3,3,19,29,8466045457
4,4,30,35,email
5,5,36,57,lictsrikant@gmail.com
6,6,58,62,life
7,7,63,72,insurance
8,8,73,84,corporation
9,9,85,87,of


In [37]:
right_table=pd.DataFrame(docjson['ents'])[['start','label']]
datafram_token=pd.merge(datafram_token,right_table,how='left',on='start')

In [40]:
datafram_token.fillna('O',inplace=True)

In [41]:
datafram_token

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,te,O
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O
...,...,...,...,...,...
57,57,448,455,promote,O
58,58,456,460,your,O
59,59,461,469,business,O
60,60,470,476,online,O


In [50]:
#join label to df_clean dataframe
df_clean['end']=df_clean['text'].apply(lambda x:len(x)+1).cumsum()-1
df_clean['start']=df_clean[['text','end']].apply(lambda x:x[1]-len(x[0]),axis=1)

C:\Users\VENUS\AppData\Local\Temp\ipykernel_46816\4235029506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['end']=df_clean['text'].apply(lambda x:len(x)+1).cumsum()-1
C:\Users\VENUS\AppData\Local\Temp\ipykernel_46816\4235029506.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['start']=df_clean[['text','end']].apply(lambda x:x[1]-len(x[0]),axis=1)


In [52]:
df_clean.head(2)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start
12,5,1,3,1,1,1,722,53,64,28,90,cell,4,0
14,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5


In [55]:
#inner join with start
dataframe_info=pd.merge(df_clean,datafram_token[['start','token','label']],how='inner',on='start')

In [57]:
dataframe_info.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,3,1,1,1,722,53,64,28,90,cell,4,0,cell,O
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,B-PHONE
2,5,1,3,2,1,1,55,55,85,89,31,te,18,16,te,O
3,5,1,3,2,1,2,822,71,203,65,96,8466045457,29,19,8466045457,B-PHONE
4,5,1,3,2,2,1,593,136,93,25,89,email,35,30,email,O


# BoundingBox

In [58]:
bb_df=dataframe_info.query("label!='O'")

In [61]:
bb_df.head(10)
img=image.copy()

In [73]:
for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x=int(x)
    y=int(y)
    w=int(w)
    h=int(h)
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [74]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,B-PHONE
3,5,1,3,2,1,2,822,71,203,65,96,8466045457,29,19,8466045457,B-PHONE
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,B-EMAIL
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,B-ORG
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,I-ORG
8,5,1,3,3,1,3,183,170,117,14,96,corporation,84,73,corporation,I-ORG
9,5,1,3,3,1,4,306,170,20,14,96,of,87,85,of,I-ORG
10,5,1,3,3,1,5,332,170,42,14,96,india,93,88,india,I-ORG
11,5,1,3,3,1,6,668,166,119,37,92,seosrika,102,94,seosrika,B-EMAIL
12,5,1,3,3,1,7,791,165,161,44,92,ntht@gmail,113,103,ntht@gmail,I-EMAIL


In [76]:
bb_df['label']=bb_df['label'].apply(lambda x:x[2:])

C:\Users\VENUS\AppData\Local\Temp\ipykernel_46816\1797870969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['label']=bb_df['label'].apply(lambda x:x[2:])


In [77]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,PHONE
3,5,1,3,2,1,2,822,71,203,65,96,8466045457,29,19,8466045457,PHONE
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,ORG
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,ORG
8,5,1,3,3,1,3,183,170,117,14,96,corporation,84,73,corporation,ORG
9,5,1,3,3,1,4,306,170,20,14,96,of,87,85,of,ORG
10,5,1,3,3,1,5,332,170,42,14,96,india,93,88,india,ORG
11,5,1,3,3,1,6,668,166,119,37,92,seosrika,102,94,seosrika,EMAIL
12,5,1,3,3,1,7,791,165,161,44,92,ntht@gmail,113,103,ntht@gmail,EMAIL


In [78]:
class groupgen():
    def __init__(self):
        self.id=0
        self.text=''
    def getgroup(self,text):
        if self.text==text:
            return self.id
        else:
            self.id+=1
            self.text=text
            return self.id

In [79]:
groupg=groupgen()

In [121]:
bb_df['group']=bb_df['label'].apply(groupg.getgroup)

C:\Users\VENUS\AppData\Local\Temp\ipykernel_46816\949760409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['group']=bb_df['label'].apply(groupg.getgroup)


In [95]:
bb_df[['left','top','width','height']]=bb_df[['left','top','width','height']].astype(int)

C:\Users\VENUS\AppData\Local\Temp\ipykernel_46816\3750294523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df[['left','top','width','height']]=bb_df[['left','top','width','height']].astype(int)


In [96]:
bb_df['right']=bb_df['left']+bb_df['width']
bb_df['bottom']=bb_df['top']+bb_df['height']

C:\Users\VENUS\AppData\Local\Temp\ipykernel_46816\3092904115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['right']=bb_df['left']+bb_df['width']
C:\Users\VENUS\AppData\Local\Temp\ipykernel_46816\3092904115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['bottom']=bb_df['top']+bb_df['height']


In [97]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group,right,bottom
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,PHONE,1,1025,81
3,5,1,3,2,1,2,822,71,203,65,96,8466045457,29,19,8466045457,PHONE,1,1025,136
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL,2,1025,167
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,ORG,3,79,184
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,ORG,3,176,193
8,5,1,3,3,1,3,183,170,117,14,96,corporation,84,73,corporation,ORG,3,300,184
9,5,1,3,3,1,4,306,170,20,14,96,of,87,85,of,ORG,3,326,184
10,5,1,3,3,1,5,332,170,42,14,96,india,93,88,india,ORG,3,374,184
11,5,1,3,3,1,6,668,166,119,37,92,seosrika,102,94,seosrika,EMAIL,4,787,203
12,5,1,3,3,1,7,791,165,161,44,92,ntht@gmail,113,103,ntht@gmail,EMAIL,4,952,209


In [98]:
#Tagging group by group
col_group=['left','top','right','bottom','label','token','group']
group_tag_img=bb_df[col_group].groupby(by='group')


In [99]:
group_tag_img.head(10)

,left,top,right,bottom,label,token,group
1,822,53,1025,81,PHONE,8099948528,1
3,822,71,1025,136,PHONE,8466045457,1
5,709,136,1025,167,EMAIL,lictsrikant@gmail.com,2
6,46,170,79,184,ORG,life,3
7,85,151,176,193,ORG,insurance,3
8,183,170,300,184,ORG,corporation,3
9,306,170,326,184,ORG,of,3
10,332,170,374,184,ORG,india,3
11,668,166,787,203,EMAIL,seosrika,4
12,791,165,952,209,EMAIL,ntht@gmail,4


In [100]:
img_tagging=group_tag_img.agg({
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x:" ".join(x)
    
})

In [101]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,822,1025,53,136,[PHONE],8099948528 8466045457
2,709,1025,136,167,[EMAIL],lictsrikant@gmail.com
3,46,374,151,193,[ORG],life insurance corporation of india
4,668,1024,165,209,[EMAIL],seosrika ntht@gmail .com
5,310,755,227,259,[NAME],thathineni srikanth
6,399,669,271,296,[DES],insurance advisor
7,47,882,395,427,[ORG],life insurance corporation of india
8,46,917,506,533,[WEB],lictsrikant8099948528.blogspot.in interviewsin...


In [102]:
img_tagging.values

array([[822, 1025, 53, 136, array(['PHONE'], dtype=object),
        '8099948528 8466045457'],
       [709, 1025, 136, 167, array(['EMAIL'], dtype=object),
        'lictsrikant@gmail.com'],
       [46, 374, 151, 193, array(['ORG'], dtype=object),
        'life insurance corporation of india'],
       [668, 1024, 165, 209, array(['EMAIL'], dtype=object),
        'seosrika ntht@gmail .com'],
       [310, 755, 227, 259, array(['NAME'], dtype=object),
        'thathineni srikanth'],
       [399, 669, 271, 296, array(['DES'], dtype=object),
        'insurance advisor'],
       [47, 882, 395, 427, array(['ORG'], dtype=object),
        'life insurance corporation of india'],
       [46, 917, 506, 533, array(['WEB'], dtype=object),
        'lictsrikant8099948528.blogspot.in interviewsinhyderabad.blogspot.in']],
      dtype=object)

In [106]:
img_bb=img.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    cv2.putText(img_bb,str(label),(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0))

In [111]:
#Parser
def parser(text,label):
    if label=='PHONE':
        text=text.lower()
        text=re.sub(r'\D','',text)
    elif label=='EMAIL':
        text=text.lower()
        allow_special_char='@_.\-'
        text=re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
    elif label=='WEB':
        text=text.lower()
        allow_special_char=':/.%#\-'
        text=re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
    elif label in ('NAME','DES'):
        text=text.lower()
        text=re.sub(r'[^a-z ]','',text)
        text=text.title()
    elif label=='ORG':
        text=text.lower()
        text=re.sub(r'[^a-z0-9]','',text)
        text=text.title()
    return text
    

In [113]:
parser('sohamtripathy2001@gmail^.com','EMAIL')

'sohamtripathy2001@gmail.com'

# Entities

In [115]:
info_array=dataframe_info[['token','label']].values

In [116]:
info_array

array([['cell', 'O'],
       ['8099948528', 'B-PHONE'],
       ['te', 'O'],
       ['8466045457', 'B-PHONE'],
       ['email', 'O'],
       ['lictsrikant@gmail.com', 'B-EMAIL'],
       ['life', 'B-ORG'],
       ['insurance', 'I-ORG'],
       ['corporation', 'I-ORG'],
       ['of', 'I-ORG'],
       ['india', 'I-ORG'],
       ['seosrika', 'B-EMAIL'],
       ['ntht@gmail', 'I-EMAIL'],
       ['.com', 'I-EMAIL'],
       ['thathineni', 'B-NAME'],
       ['srikanth', 'I-NAME'],
       ['insurance', 'B-DES'],
       ['advisor', 'I-DES'],
       ['agent', 'O'],
       ['code', 'O'],
       ['no', 'O'],
       ['0316164y', 'O'],
       ['life', 'B-ORG'],
       ['insurance', 'I-ORG'],
       ['corporation', 'I-ORG'],
       ['of', 'I-ORG'],
       ['india', 'I-ORG'],
       ['br', 'O'],
       ['off', 'O'],
       ['lic', 'O'],
       ['office', 'O'],
       ['trimulgherry', 'O'],
       ['sec’bad', 'O'],
       ['-', 'O'],
       ['500', 'O'],
       ['016', 'O'],
       ['add', 'O'],
       [

In [119]:
entities=dict(NAME=[],ORG=[],DES=[],PHONE=[],EMAIL=[],WEB=[])
previous='O'


for token,label in info_array:
    bio_tag = label[0]
    label_tag=label[2:]
    #step-1 parse the  text
    text=parser(token,label_tag)
    if bio_tag in ('B','I'):
        if previous!=label_tag:
            entities[label_tag].append(text)
        else:
            if bio_tag=='B':
                entities[label_tag].append(text)
            else:
                if label_tag in ("NAME",'ORG','DES'):
                    entities[label_tag][-1]=entities[label_tag][-1]+" "+text
                else:
                    entities[label_tag][-1]=entities[label_tag][-1]+text
    previous=label_tag
            
    

In [120]:
print(entities)

{'NAME': ['Thathineni Srikanth'], 'ORG': ['Life Insurance Corporation Of India', 'Life Insurance Corporation Of India'], 'DES': ['Insurance Advisor'], 'PHONE': ['8099948528', '8466045457'], 'EMAIL': ['lictsrikant@gmail.com', 'seosrikantht@gmail.com'], 'WEB': ['lictsrikant8099948528.blogspot.in', 'interviewsinhyderabad.blogspot.in']}
